# DCNN test

DCNN test notebook is a place for exploring customized layers of dcnn with tensorflow.

In [1]:
import numpy as np
import tensorflow as tf

*Do Experinemts on the padding function on tensorflow*

In [2]:
t = tf.constant([[1, 2, 3], [4, 5, 6]])

# Set up the padding values
# [1, 1]: Add 1 pad for "height(row)" dimension
# [2, 2]: Add 2 pads for "weight(col)" dimension
paddings = tf.constant([[1, 1,], [2, 2]])

t_pad = tf.pad(t, paddings, "CONSTANT")

with tf.Session() as sess:
    
    r = sess.run(t_pad)
    
    print(r)

[[0 0 0 0 0 0 0]
 [0 0 1 2 3 0 0]
 [0 0 4 5 6 0 0]
 [0 0 0 0 0 0 0]]


In [3]:
# Set up the padding values asymmetric 
# [1, 2]: Prepend 1 pad before the values and append 2 pads after the values in dimension 0
# [3, 4]: Prepend 3 pads befroe the values and append 4 pads after the values in dimension 1
paddings = tf.constant([[1, 2], [3, 4]])

t_pad = tf.pad(t, paddings, "CONSTANT")

with tf.Session() as sess:
    
    print(sess.run(t_pad))


[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 2 3 0 0 0 0]
 [0 0 0 4 5 6 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


In [4]:
t_4d = tf.constant([[[[1, 2, 3], [4, 5, 6]]]])
paddings = tf.constant([[0,0], [0,0], [1, 1], [2, 2]])

t_pad = tf.pad(t_4d, paddings, "CONSTANT")

with tf.Session() as sess:
    
    r = sess.run(t_pad)
    print(r.shape)
    print(r)

(1, 1, 4, 7)
[[[[0 0 0 0 0 0 0]
   [0 0 1 2 3 0 0]
   [0 0 4 5 6 0 0]
   [0 0 0 0 0 0 0]]]]


## Test: KMaxPooling2D

KmaxPooling is a fundamental layer of dynamic cnn. 

#### Op implementation

In [35]:
import math

# 20180604 Lin, Y.D.: Read how tensorflow does padding dynamically. 
def _kmax_pooling2d(X, pool_size, strides, k, padding):
    print('_kmax_pooling2d starts')
    h_pool = pool_size[0]
    w_pool = pool_size[1]

    h_stride = strides[0]
    w_stride = strides[1]
    
    h_pad = padding[0]
    w_pad = padding[1]
    
    # 20180715 LIN, Y.D. Add batch
    batch = X.shape[0]
    h = X.shape[1]
    w = X.shape[2]
    
    # 20180719 LIN, Y.D. Add channel
    channel = X.shape[3]
    
    # 20180715 LIN, Y.D. Reshaped seems no need with batch argument
#     X = np.reshape(X, (h, w, 1))
    
    # 20180613 LIN, Y.D. padding has been added in class KMaxPooling2D previosly. No Padding need here
    if h_stride == 0:
        outputs_h = 1
    else:
        outputs_h = math.ceil((h-h_pool+1)/h_stride)
        
    if w_stride == 0:
        outputs_w = k
    else:
        outputs_w = k * math.ceil((w-w_pool+1)/w_stride)
        
#     outputs_h = math.ceil((h-h_pool+1)/h_stride)
#     outputs_w = k * math.ceil((w-w_pool+1)/w_stride)
    
    
    # 20180714 LIN, Y.D. Add batch argument
    outputs = np.zeros((batch, outputs_h, outputs_w, channel), dtype=np.float64)
#     outputs = np.zeros((batch, outputs_h, outputs_w, 1), dtype=np.float64)
    
#     outputs = np.zeros((outputs_h, outputs_w, 1), dtype=np.float64)
    
    # 20180714 LIN, Y.D. Case for Stride 0
    if h_stride == 0:
        h_stride = h
        h_bound  = h
    else:
        h_bound = h-h_pad
        
    if w_stride == 0:
        w_stride = w
        w_bound  = w 
    else:
        w_bound = w-w_pad
    
    # 20180719 LIN, Y.D. Add channel
    for _b in range(0, batch):
        
        for _c in range(0, channel):
        
            m = 0
            for _h in range(0, h_bound, h_stride):
            
                n = 0
                for _w in range(0, w_bound, w_stride):
            
                    flatten_slice = X[_b, _h:_h+h_pool, _w:_w+w_pool, _c].ravel()
                    top_indices = np.sort(
                        np.argpartition(flatten_slice, -1*np.arange(k))[h_pool*w_pool-k: h_pool*w_pool])
                    outputs[_b, m, n:n+k, _c] = np.reshape(flatten_slice[top_indices], (1, k))
            
                    n += k
                
                    if n >= outputs_w:
                        break

                m += 1
                if m >= outputs_h:
                    break
    
    # 20180715 LIN, Y.D. Add batch
#     for _b in range(0, batch):
        
#         m = 0
#         for _h in range(0, h_bound, h_stride):
            
#             n = 0
#             for _w in range(0, w_bound, w_stride):
            
#                 flatten_slice = X[_b, _h:_h+h_pool, _w:_w+w_pool].ravel()
#                 top_indices = np.sort(
#                     np.argpartition(flatten_slice, -1*np.arange(k))[h_pool*w_pool-k: h_pool*w_pool])
    
#             # 20180714 LIN, Y.D. Discovery a Bug Here (Fixed)
# #             top_indices = np.sort(
# #                 np.argpartition(flatten_slice, -1*np.arange(k))[h_pool+w_pool-k: h_pool+w_pool])
#                 outputs[_b, m, n:n+k] = np.reshape(flatten_slice[top_indices], (1, k, 1))
            
#                 n += k
                
#                 if n >= outputs_w:
#                     break

#             m += 1
#             if m >= outputs_h:
#                 break  
            
    return outputs

    

#### Layer Implementation

In [42]:
#20180607 LIN, Y.D. 
class KMaxPooling2D(tf.layers.Layer):
    
    def __init__(self, pool_size, strides, k, padding='valid', name=None, **kwargs):
        
        super(KMaxPooling2D, self).__init__(name=name, **kwargs)
        
        if padding != 'same' and padding != 'valid':
            raise ValueError('Invalid Padding Options. Only SAME and VALID are valid.')
            
        # 20180610 LIN, Y.D. Add the exception when 
        if strides[0] == 0 and strides[1] == 0:
            raise ValueError('Invalid Stride Values: At least 1 stride value should be greater than 0.')

        self.k = k
        self.padding = padding
        self.pool_size = pool_size
        self.strides = strides
    
    def call(self, inputs):
        print('call start')
        inputs_shape = inputs.get_shape().as_list()
        
        if len(inputs_shape) < 4:
            raise ValueError('The input should be a 4-D tensor')
        
        h = inputs_shape[1]
        w = inputs_shape[2]
        
        if inputs_shape == [None, None, None, None]:
            return [tf.constant([0]), tf.constant([0]), tf.constant([0]), tf.constant([0])]
        
        h_stride = self.strides[0]
        w_stride = self.strides[1]
        
        h_pool_s = self.pool_size[0]
        w_pool_s = self.pool_size[1]
        
        # 20180612 LIN, Y.D. 
        pool_size = tf.constant(self.pool_size, dtype=tf.int32)
        strides   = tf.constant(self.strides, dtype=tf.int32)
        k = tf.constant(self.k, dtype=tf.int32)
        padding = tf.constant([0, 0], dtype=tf.int32)
        
        if self.padding == 'valid':
            if ((h - h_pool_s) / h_stride + 1) % 1 != 0:
                raise ValueError('The height is invalid')
            if ((w - w_pool_s) / w_stride + 1) % 1 != 0:
                raise ValueError('The weight is invalid')
        
        elif self.padding == 'same':
            
            # 20180713 LIN, Y.D. h_stride/w_stride is 0 case
            if h_stride == 0:
                h_pad = 0
            else:
                h_mod = h % h_stride
                
                if h_mod == 0:
                    h_pad = max([h_pool_s - h_stride, 0])
                else:
                    h_pad = max([h_pool_s - h_mod, 0])
                    
            if w_stride == 0:
                w_pad = 0
            else:
                w_mod = w % w_stride
            
                if w_mod == 0:
                    w_pad = max([w_pool_s - w_stride, 0])
                else:
                    w_pad = max([w_pool_s - w_mod, 0])
        
            # 20180606 Y.D. Padding input with 0.
            h_pad_mod_2 = h_pad % 2
            w_pad_mod_2 = w_pad % 2
        
            if h_pad_mod_2 == 0:
                h_pad_top, h_pad_bottom = h_pad//2, h_pad//2
            else:
                h_pad_top, h_pad_bottom = h_pad//2, h_pad//2 + h_pad_mod_2
    
            if w_pad_mod_2 == 0:
                w_pad_left, w_pad_right = w_pad//2, w_pad//2
            else:
                w_pad_left, w_pad_right = w_pad//2, w_pad//2 + w_pad_mod_2
            
            pad_vals = tf.constant(
                [[0, 0], [h_pad_top, h_pad_bottom], [w_pad_left, w_pad_right], [0, 0]], 
                dtype=tf.int32)
        
            inputs = tf.pad(inputs, pad_vals, "CONSTANT")
            
            # 20180612 LIN, Y.D. Update inputs
            padding = tf.constant([h_pad_top+h_pad_bottom, w_pad_left+w_pad_right], dtype=tf.int32)
        
        print('Prepare to enter _kmax_pooling2d')
        outputs = tf.py_func(_kmax_pooling2d, [inputs, pool_size, strides, k, padding], tf.float64)
        
        self.compute_output_shape(inputs_shape)
        
        return outputs
    
    def compute_output_shape(self, input_shape):
        
        print('Computing output shape')
        return input_shape

In [43]:
def kmax_pooling2d(inputs, pool_size, strides, k, padding, name=None):
    return KMaxPooling2D(pool_size, strides, k, padding=padding, name=name).apply(inputs)

In [20]:
# 20180605 LIN, Y.D.
with tf.Session() as sess:
    
    test_x = np.array([[[1,1,3], [3,1,2]]], dtype=np.float64)
    test_input = tf.placeholder(tf.float64, (1, 2, 3))
    test_reshape = tf.reshape(test_input, shape=(1,2,3,1))
    
    # Test Case 1: 
    test_layer = kmax_pooling2d(test_reshape, [2, 2], [1, 1], 2, 'same')
    res = sess.run(test_layer, feed_dict={ test_input: test_x })
    print(res)
    print(res.shape)
    print('-' * 10)
    
    # Test Case 2: Test if exception work.
    try:
        test_layer_2 = kmax_pooling2d(test_reshape, [3, 3], [0, 0], 2, 'same')
        res = sess.run(test_layer_2, feed_dict={ test_input: test_x })
        print(res)
        print(res.shape)
    except ValueError as e:
        print('Error is caught successfully')
        print(e)
        pass
    print('-' * 10)
    
#     # Test Case 3: Test Valid
    test_x_3 = np.array([[[1, 2, 3], [3, 1, 2], [2, 1, 3]]], dtype=np.float64)
    test_input_3 = tf.placeholder(tf.float64, (1, 3, 3))
    test_reshape_3 = tf.reshape(test_input_3, (1, 3, 3, 1))
    test_layer_3 = kmax_pooling2d(test_reshape_3, [2, 2], [1, 1], 2, 'valid')
    res = sess.run(test_layer_3, feed_dict={ test_input_3: test_x_3 })
    print(res)
    print(res.shape)
    print('-' * 10)
    
    # Test Case 4: If the first stride will cross over the border.
    test_layer_4 = kmax_pooling2d(test_reshape_3, [3, 3], [3, 3], 2, 'same')
    res = sess.run(test_layer_4, feed_dict={ test_input_3: test_x_3 })
    print(res)
    print(res.shape)
    print('-' * 10)
    
    # Test Case 5: Continue the previous test, we use valid as padding policy
    test_layer_5 = kmax_pooling2d(test_reshape_3, [3, 3], [3, 3], 2, 'valid')
    res = sess.run(test_layer_5, feed_dict={ test_input_3: test_x_3 })
    print(res)
    print(res.shape)
    print('-' * 10)
    
    # Test Case 6: Filter should not bigger than the input matrix
    test_layer_6 = kmax_pooling2d(test_reshape_3, [4, 4], [2, 2], 2, 'same')
    res = sess.run(test_layer_6, feed_dict={ test_input_3: test_x_3 })
    print(res)
    print(res.shape)
    print('-' * 10)
    
    # Test Case 7:  
    try:
        test_layer_7 = kmax_pooling2d(test_reshape_3, [4, 4], [2, 2], 2, 'valid')
        res = sess.run(test_layer_7, feed_dict={ test_input_3: test_x_3 })
        print(res)
        print(res.shape)
        print('-' * 10)
    except ValueError as e:
        print(e)
        print('Error is caught successfully')
        
    # Test Case 8: Stride only on width direction
    test_layer_8 = kmax_pooling2d(test_reshape_3, [3, 1], [0, 1], 2, 'same')
    res = sess.run(test_layer_8, feed_dict={ test_input_3: test_x_3 })
    print('Test Case 8')
    print(res)
    print(res.shape)
    print('-' * 10)
    
    # Test Case 9: Stride only on height direction
    test_layer_9 = kmax_pooling2d(test_reshape_3, [1, 3], [1, 0], 2, 'same')
    res = sess.run(test_layer_9, feed_dict={ test_input_3: test_x_3 })
    print('Test Case 9:')
    print(res)
    print(res.shape)
    print('-' * 10)
    
#     # Test Case 10: Batch
    test_x_4 = np.array([
        [[1, 2, 3], [3, 1, 2], [2, 1, 3]],
        [[1, 2, 3], [3, 1, 2], [2, 1, 3]]], dtype=np.float64)
    test_input_4 = tf.placeholder(tf.float64, (2, 3, 3))
    test_reshape_4 = tf.reshape(test_input_4, (2, 3, 3, 1))
    test_layer_10 = kmax_pooling2d(test_reshape_4, [1, 3], [1, 0], 2, 'same')
    res = sess.run(test_layer_10, feed_dict={ test_input_4: test_x_4 })
    print('Test Case 10:')
    print(res)
    print(res.shape)
    print('-' * 10)

[[[[3.]
   [1.]
   [3.]
   [2.]
   [3.]
   [2.]]

  [[3.]
   [1.]
   [1.]
   [2.]
   [2.]
   [0.]]]]
(1, 2, 6, 1)
----------
Error is caught successfully
Invalid Stride Values: At least 1 stride value should be greater than 0.
----------
[[[[2.]
   [3.]
   [3.]
   [2.]]

  [[3.]
   [2.]
   [2.]
   [3.]]]]
(1, 2, 4, 1)
----------
[[[[1.]
   [3.]]]]
(1, 1, 2, 1)
----------
[[[[1.]
   [3.]]]]
(1, 1, 2, 1)
----------
[[[[1.]
   [3.]
   [3.]
   [0.]]

  [[3.]
   [0.]
   [3.]
   [0.]]]]
(1, 2, 4, 1)
----------
The height is invalid
Error is caught successfully
Test Case 8
[[[[3.]
   [2.]
   [2.]
   [1.]
   [3.]
   [3.]]]]
(1, 1, 6, 1)
----------
Test Case 9:
[[[[2.]
   [3.]]

  [[3.]
   [2.]]

  [[2.]
   [3.]]]]
(1, 3, 2, 1)
----------
Test Case 10:
[[[[2.]
   [3.]]

  [[3.]
   [2.]]

  [[2.]
   [3.]]]


 [[[2.]
   [3.]]

  [[3.]
   [2.]]

  [[2.]
   [3.]]]]
(2, 3, 2, 1)
----------


#### Train a simple model with kmaxpooling


In [21]:
num_words = 100
maxlen = 100
batch_size = 32

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=num_words)

In [22]:
padded_x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen, padding='pre')
padded_x_test  = tf.keras.preprocessing.sequence.pad_sequences(x_test,  maxlen=maxlen, padding='pre')

In [44]:
input_x = tf.placeholder(tf.int64, (batch_size, maxlen))
one_hot = tf.one_hot(input_x, 100)
conv = tf.layers.conv1d(
    one_hot, 64, kernel_size=(100,), strides=1, padding='same',
    kernel_initializer=tf.truncated_normal_initializer(stddev=5e-2, dtype=tf.float32),
    bias_initializer=tf.truncated_normal_initializer(stddev=5e-2, dtype=tf.float32)
)

reshape = tf.reshape(conv, [batch_size, maxlen, 1, 64])
kmax = kmax_pooling2d(reshape, [100, 1], [0, 1], 2, 'same')
kmax.set_shape([32, 1, 2, 64])
flatten = tf.layers.flatten(kmax)

init = tf.global_variables_initializer()

# with tf.Session() as sess:
#     sess.run(init)
#     res = sess.run(one_hot, feed_dict={ input_x: padded_x_train[:32] })
#     print('one hot:', res.shape)
#     print(res)
#     res = sess.run(conv, feed_dict={ one_hot: res })
#     print('shape of conv1d:', res.shape)
#     print(res)
#     res = sess.run(reshape, feed_dict={ conv: res })
#     print('reshape of conv1d:')
#     print(res.shape)
#     res = sess.run(kmax, feed_dict={ reshape: res })
#     print('kmax:', res.shape)
#     print(res)
#     res = sess.run(flatten, feed_dict={ kmax: res} )
#     print('flatten', res.shape)
#     print(res)

call start
Prepare to enter _kmax_pooling2d
Computing output shape


In [54]:
def conv_net(features, reuse, dropout, is_training):
    
    with tf.variable_scope('conv_net', reuse=reuse):
#         input_x = tf.placeholder(tf.int64, (batch_size, features))
#         reshape = tf.reshape(features, shape=[-1, 100, 1, 1])
        one_hot = tf.one_hot(features, 100)
        conv = tf.layers.conv1d(
            one_hot, 64, kernel_size=(100,), strides=1, padding='same',
            kernel_initializer=tf.truncated_normal_initializer(stddev=5e-2, dtype=tf.float32),
            bias_initializer=tf.truncated_normal_initializer(stddev=5e-2, dtype=tf.float32)
        )

        conv = tf.reshape(conv, [batch_size, maxlen, 1, 64])
        kmax = kmax_pooling2d(conv, [100, 1], [0, 1], 2, 'same')
#         kmax = kmax_pooling2d(reshape, [100, 1], [0, 1], 2, 'same')
        fc = tf.contrib.layers.flatten(kmax)
        fc = tf.layers.dense(fc, 100)
        fc = tf.layers.dropout(fc, rate=dropout, training=is_training)
        output = tf.layers.dense(fc, 2)
    
    return output

def model_fn(features, labels, mode):
    
    logits_train = conv_net(features, False, 0.4, is_training=True)
    logits_test = conv_net(features, True, 0.4, is_training=False)
    
    pred_classes = tf.argmax(logits_test, axis=1)
    pred_probs = tf.nn.softmax(logits_test)
    
    if model == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)
    
    loss_op = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits_train), 
        labels=tf.cast(labels, dtype=tf.int32))
    optimizer = tf.train.AdamOptimizer(loss_op, global_steps=tf.train.get_global_step())
    
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    
    estim_specs = tf.estimator.EstimatorSpec(
        mode=mode, 
        predictions=pred_classes, 
        loss=loss_op, 
        eval_metric_ops={'accuracy': acc_op})
    
    return estim_specs

model = tf.estimator.Estimator(model_fn)

input_fn = tf.estimator.inputs.numpy_input_fn(
    x=padded_x_train, 
    y=y_train,
    batch_size=32,
    num_epochs=None,
    shuffle=True
)

model.train(input_fn, steps=10)

input_test_fn = tf.estimator.inputs.numpy_input_fn(
    x=padded_x_test, 
    y=y_test,
    batch_size=32,
    shuffle=False
)

e = model.evaluate(input_test_fn)

e['accuracy']

# init = tf.global_variables_initializer()

# with tf.Session() as sess:
#     sess.run(init)
#     res = sess.run(one_hot, feed_dict={ input_x: padded_x_train[:32] })
#     print('one hot:', res.shape)
#     print(res)
#     res = sess.run(conv, feed_dict={ one_hot: res })
#     print('shape of conv1d:', res.shape)
#     print(res)
#     res = sess.run(reshape, feed_dict={ conv: res })
#     print('reshape of conv1d:')
#     print(res.shape)
#     res = sess.run(kmax, feed_dict={ reshape: res })
#     print('kmax:', res.shape)
#     print(res)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/hb/3__tt48x0nvgkfy5cr7skv_c0000gn/T/tmp11lyzczu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12e2aa898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.


ValueError: Input 0 of layer flatten_1 is incompatible with the layer: its rank is undefined, but the layer requires a defined rank.

In [45]:
res[0]

array([[[-0.22882307],
        [-0.3155947 ]]])

## Test: Folding

Folding is introduced before the last KMax layer.

## Test: Stack a "non-deep" dcnn (1)

We stack a shallow dcnn to make sure no bug exists in inference stage.


## Test: Stack a "non-deep" dcnn (2)

Train a "non-deep" dcnn with varied length as input.

In [53]:
x = np.array([3, 4, 2, 1, 5, 7, 2, 6])
print(np.argpartition(x, 3))
print(x[np.argpartition(x, 3)])

[6 3 2 0 4 5 1 7]
[2 1 2 3 5 7 4 6]


In [54]:
print(np.argpartition(x, -3))
print(x[np.argpartition(x, -3)])

[6 3 2 0 1 4 7 5]
[2 1 2 3 4 5 6 7]


In [48]:
test = np.array([[[[1, 2, 3], [2, 3, 1], [3, 2, 1]], [[3, 2, 1], [1, 2, 3], [2, 3, 1]], [[2, 1, 3], [1, 2, 3], [3, 1, 2]]]])

In [51]:
test[0, 0:2, 0:2, 2]

array([[3, 1],
       [1, 3]])